In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('JaxHousingData.csv')
df.fillna(0, inplace=True)
df.head()

,RE,mailing_address,mailing_city,mailing_state,property_use,subdivision,neighborhood,property_value,county_taxable,lot_size,...,storages,storage_area,storage_effec,storage_heated,storage_unfinished,upper_stories,upper_story_area,upper_story_effec,upper_story_heated,upper_story_unfinished
0,0000060040R,2501 SUMMERFIELD LN,JACKSONVILLE,FL,200,SECTION LAND,417300.2,38078,38078,160153,...,0,0,0,0,0,0,0,0,0,0
1,0000060040R,2501 SUMMERFIELD LN,JACKSONVILLE,FL,200,SECTION LAND,417300.2,38078,38078,160153,...,0,0,0,0,0,0,0,0,0,0
2,0000060100R,5509 POTOMAC AVE,JACKSONVILLE,FL,200,SECTION LAND,417300.2,82705,82705,217395,...,0,0,0,0,0,0,0,0,0,0
3,0000070010R,PO BOX 23637,JACKSONVILLE,FL,100,SECTION LAND,417300.2,72691,72691,52759,...,0,0,0,0,0,0,0,0,0,0
4,0000070010R,PO BOX 23637,JACKSONVILLE,FL,100,SECTION LAND,417300.2,72691,72691,52759,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(58983, 182)

In [4]:
empty_cols = []
for i in range(len(df.columns)):
    print(df.columns[i])
    print(df.dtypes[i])
    if df.dtypes[i] != 'object':
        nlls = sum(df[df.columns[i]]==0)
        nll_percent = round(nlls/len(df),2)
        print('Null:', nlls, ' - ', str(nll_percent)+'%')
        if nll_percent >= 0.95:
            empty_cols.append(df.columns[i])
    print()

RE
object

mailing_address
object

mailing_city
object

mailing_state
object

property_use
int64
Null: 1736  -  0.03%

subdivision
object

neighborhood
float64
Null: 0  -  0.0%

property_value
int64
Null: 0  -  0.0%

county_taxable
int64
Null: 601  -  0.01%

lot_size
int64
Null: 0  -  0.0%

building
int64
Null: 0  -  0.0%

type_descr
object

style
int64
Null: 0  -  0.0%

class
int64
Null: 0  -  0.0%

quality
float64
Null: 0  -  0.0%

yr_built
int64
Null: 0  -  0.0%

bldng_value
int64
Null: 1845  -  0.03%

heated_sf
int64
Null: 0  -  0.0%

trans_id
object

sale_date
object

trans_price
int64
Null: 0  -  0.0%

parcel_count
int64
Null: 0  -  0.0%

ppsf
int64
Null: 57  -  0.0%

eff_price
int64
Null: 57  -  0.0%

baths
float64
Null: 0  -  0.0%

bedrooms
float64
Null: 2  -  0.0%

rooms
float64
Null: 0  -  0.0%

stories
float64
Null: 10  -  0.0%

boat_cv
float64
Null: 58665  -  0.99%

boatcv_avg_grade
float64
Null: 58665  -  0.99%

boatcv_avg_ppu
float64
Null: 58665  -  0.99%

boatcv_avg_yr_b

In [5]:
print(empty_cols)

['boat_cv', 'boatcv_avg_grade', 'boatcv_avg_ppu', 'boatcv_avg_yr_built', 'boatcv_total_depreciated', 'carport_ft', 'carportft_avg_grade', 'carportft_avg_ppu', 'carportft_avg_yr_built', 'carportft_total_depreciated', 'dock', 'dock_avg_grade', 'dock_avg_ppu', 'dock_avg_yr_built', 'dock_total_depreciated', 'elevator', 'elevator_avg_grade', 'elevator_avg_ppu', 'elevator_avg_yr_built', 'elevator_total_depreciated', 'fence', 'fence_avg_grade', 'fence_avg_ppu', 'fence_avg_yr_built', 'fence_total_depreciated', 'gazebo', 'gazebo_avg_grade', 'gazebo_avg_ppu', 'gazebo_avg_yr_built', 'gazebo_total_depreciated', 'greenhouse', 'greenhouse_avg_grade', 'greenhouse_avg_ppu', 'greenhouse_avg_yr_built', 'greenhouse_total_depreciated', 'sauna', 'sauna_avg_grade', 'sauna_avg_ppu', 'sauna_avg_yr_built', 'sauna_total_depreciated', 'shed', 'shed_avg_grade', 'shed_avg_ppu', 'shed_avg_yr_built', 'shed_total_depreciated', 'spa', 'spa_avg_grade', 'spa_avg_ppu', 'spa_avg_yr_built', 'spa_total_depreciated', 'sun_rm

In [6]:
df = df.drop(columns=empty_cols)
df.shape

(58983, 72)

In [7]:
for i in range(len(df.columns)):
    print(df.columns[i])
    print(df.dtypes[i])
    print()

RE
object

mailing_address
object

mailing_city
object

mailing_state
object

property_use
int64

subdivision
object

neighborhood
float64

property_value
int64

county_taxable
int64

lot_size
int64

building
int64

type_descr
object

style
int64

class
int64

quality
float64

yr_built
int64

bldng_value
int64

heated_sf
int64

trans_id
object

sale_date
object

trans_price
int64

parcel_count
int64

ppsf
int64

eff_price
int64

baths
float64

bedrooms
float64

rooms
float64

stories
float64

cov_patio
float64

covpatio_avg_grade
float64

covpatio_avg_ppu
float64

covpatio_avg_yr_built
float64

covpatio_total_depreciated
float64

deck_ft
float64

deckft_avg_grade
float64

deckft_avg_ppu
float64

deckft_avg_yr_built
float64

deckft_total_depreciated
float64

fireplace
float64

fireplace_avg_grade
float64

fireplace_avg_ppu
float64

fireplace_avg_yr_built
float64

fireplace_total_depreciated
float64

pool
float64

pool_avg_grade
float64

pool_avg_ppu
float64

pool_avg_yr_built
float64

p

In [8]:
df = df.astype(
    {'property_use':'object',
     'neighborhood':'object',
     'yr_built':'int64'}
)

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
property_value,58983.0,183359.928895,155598.669954,2644.0,101043.0,159580.0,224504.5,4574034.0
county_taxable,58983.0,150862.753912,144285.186044,0.0,72786.5,126816.0,186118.5,4574034.0
lot_size,58983.0,12811.532713,27880.244335,766.0,6114.5,8471.0,11969.5,2643732.0
building,58983.0,1.013122,0.179189,1.0,1.0,1.0,1.0,21.0
style,58983.0,1.021769,0.145930,1.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...
storage_unfinished,58983.0,0.137667,0.378237,0.0,0.0,0.0,0.0,4.0
upper_stories,58983.0,0.273503,0.554693,0.0,0.0,0.0,0.0,10.0
upper_story_area,58983.0,230.662564,469.436528,0.0,0.0,0.0,0.0,5011.0
upper_story_effec,58983.0,219.612092,447.140821,0.0,0.0,0.0,0.0,4760.0


In [10]:
df.to_csv('JaxHousingData.csv')